In [41]:
import requests
import numpy as np
import pandas as pd
import sys
from io import BytesIO
import json
import os
import re

Data set

In [42]:
f = open('./translate\SemEval2010_task8_all_data\SemEval2010_task8_training/TRAIN_FILE.txt')

sentence_list = []

relation_list = []

comment_list = []
    
n = 0

for line in f.readlines():
    
    n += 1
    
    if n % 4 ==0:
        pass

    if n % 4 ==1:
        m = re.compile('["]').search(line).start()
        line = line[m:].replace('"','').strip()
        sentence_list.append(line)
        
    if n % 4 ==2:
        relation_list.append(line.strip())
        
    if n % 4 ==3:
        comment_list.append(line.strip())
    

train_df = pd.DataFrame({'sentence': sentence_list,          
                   'relation': relation_list, 
                   'commnet': comment_list},
                 )

In [43]:
train_df.head(5)

,sentence,relation,commnet
0,The system as described above has its greatest...,"Component-Whole(e2,e1)",Comment: Not a collection: there is structure ...
1,The <e1>child</e1> was carefully wrapped and b...,Other,Comment:
2,The <e1>author</e1> of a keygen uses a <e2>dis...,"Instrument-Agency(e2,e1)",Comment:
3,A misty <e1>ridge</e1> uprises from the <e2>su...,Other,Comment:
4,The <e1>student</e1> <e2>association</e2> is t...,"Member-Collection(e1,e2)",Comment:


In [44]:
f = open('./translate\SemEval2010_task8_all_data\SemEval2010_task8_testing/TEST_FILE.txt')

sentence_list = []

for line in f.readlines():
    m = re.compile('["]').search(line).start()
    line = line[m:].replace('"','').strip()
    sentence_list.append(line)    

test_df = pd.DataFrame({'sentence': sentence_list})

In [45]:
test_df.head()

,sentence
0,The most common <e1>audits</e1> were about <e2...
1,The <e1>company</e1> fabricates plastic <e2>ch...
2,The school <e1>master</e1> teaches the lesson ...
3,The suspect dumped the dead <e1>body</e1> into...
4,Avian <e1>influenza</e1> is an infectious dise...


In [46]:
f = open('./translate\Portuguese/DBpediaRelations-PT-0.2.txt',encoding='utf8')
sentence_list = []
mc_list = []
e1_list = [] 
type1_list = []
e2_list = []
type2_list = []
rel_type_list = []

for line in f.readlines():
    if 'SENTENCE' in line:
        m = re.compile('SENTENCE').search(line).end()
        line = line[m+2:].replace('"','').strip()
        sentence_list.append(line)
        
    if 'MANUALLY CHECKED' in line:
        m = re.compile('MANUALLY CHECKED').search(line).end()
        line = line[m+2:].replace('"','').strip()
        mc_list.append(line)
        
    if 'ENTITY1' in line:
        m = re.compile('ENTITY1').search(line).end()
        line = line[m+2:].replace('"','').strip()
        e1_list.append(line)
        
    if 'TYPE1' in line:
        m = re.compile('TYPE1').search(line).end()
        line = line[m+2:].replace('"','').strip()
        type1_list.append(line)
        
    if 'ENTITY2' in line:
        m = re.compile('ENTITY2').search(line).end()
        line = line[m+2:].replace('"','').strip()
        e2_list.append(line)
        
    if 'TYPE2' in line:
        m = re.compile('TYPE2').search(line).end()
        line = line[m+2:].replace('"','').strip()
        type2_list.append(line)
        
    if 'REL TYPE' in line:
        m = re.compile('REL TYPE').search(line).end()
        line = line[m+2:].replace('"','').strip()
        rel_type_list.append(line)

polt_df = pd.DataFrame({'sentence': sentence_list,
                        'manually checked': mc_list,
                        'entitiy1': e1_list,
                        'type1': type1_list,
                        'entity2': e2_list,
                        'type2': type2_list,
                        'relation': rel_type_list}
                      )
# 647 번째 줄인가가 TYPE2 가 TYPE1로 되어있음

In [47]:
polt_df.head(5)

,sentence,manually checked,entitiy1,type1,entity2,type2,relation
0,A América Latina localiza-se totalmente no hem...,TRUE,América Latina,LOCATION,Argentina,LOCATION,locatedInArea
1,"A América Latina não apresenta, ao contrário d...",TRUE,América Latina,LOCATION,Brasil,LOCATION,locatedInArea
2,A América Latina destaca-se ainda por sua prod...,TRUE,América Latina,LOCATION,Brasil,LOCATION,locatedInArea
3,Albert Einstein nasceu na região alemã de Würt...,TRUE,Albert Einstein,PERSON,Ulm,LOCATION,origin
4,Albert Einstein nasceu na região alemã de Würt...,TRUE,Albert Einstein,PERSON,Württemberg,LOCATION,origin


Translate function(Use Kakao API)

In [48]:
MYAPP_KEY = '002558914cc8caacc923bbb4f9d6a772'
header = {
        "Authorization" : 'KakaoAK {}'.format(MYAPP_KEY)
    }

In [49]:
url = 'https://kapi.kakao.com/v1/translation/translate'

# 002558914cc8caacc923bbb4f9d6a772 : 준원
# 03e207126164816f438e90f73832984c : 소연
# 18fea080a0db2ee0346353e8db4466e8 : 윤전
before_lang = input()
after_lang = input()
#string = 
'''
    지난해 3월 오픈한 카카오톡 주문하기는 현재까지 약 250만명의 회원을 확보했으며, 주문 가능한 프랜차이즈 브랜드는 38개,
    가맹점수는 약 1만 5천여곳에 달한다. 전 국민에게 친숙한 카카오톡 UI를 활용하기 때문에 남녀노소 누구나 쉽게 이용할 수 있으며,
    별도의 앱을 설치할 필요 없이 카카오톡 내에서 모든 과정이 이뤄지는 것이 특징이다. 지난해 9월 업계 최초로 날짜와 시간을 예약한 뒤
    설정한 매장에서 주문 음식을 찾아가는 ‘픽업’ 기능을 도입했고, 올해 1월 스마트스피커 ‘카카오미니’에서 음성을 통해 주문 가능한
    메뉴를 안내받을 수 있도록 서비스를 연동하며 차별화를 꾀했다. 중소사업자들이 카카오톡 주문하기에 입점하게 되면 4,300만 카카오톡
    이용자들과의 접점을 확보하고, 간편한 주문 과정으로 만족도를 높일 수 있게 된다. 카카오톡 메시지를 통해 신메뉴 출시, 프로모션 등의
    소식을 전달할 수 있고, 일대일 채팅 기능을 적용하면 고객과 직접 상담도 가능하다.
'''

header = {
        "Authorization" : 'KakaoAK {}'.format(MYAPP_KEY)
    }

data = {
        "src_lang": before_lang,
        "target_lang": after_lang,
        "query": string
    }

# response = requests.get(url,headers=header, params=data)
# result = response.json()
# print(result['translated_text'][0])

kr
en


In [50]:
def translate(string,before_lang, after_lang):
    response = requests.get(url,headers=header, params=data)
    result = response.json()
    a = ''
    for _ in result['translated_text'][0]:
        a = a + _
    
    return a

In [40]:
translate(string, 'en', 'kr')

ConnectionError: HTTPSConnectionPool(host='kapi.kakao.com', port=443): Max retries exceeded with url: /v1/translation/translate?src_lang=kr&target_lang=en&query=%0A++++%EC%A7%80%EB%82%9C%ED%95%B4+3%EC%9B%94+%EC%98%A4%ED%94%88%ED%95%9C+%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%86%A1+%EC%A3%BC%EB%AC%B8%ED%95%98%EA%B8%B0%EB%8A%94+%ED%98%84%EC%9E%AC%EA%B9%8C%EC%A7%80+%EC%95%BD+250%EB%A7%8C%EB%AA%85%EC%9D%98+%ED%9A%8C%EC%9B%90%EC%9D%84+%ED%99%95%EB%B3%B4%ED%96%88%EC%9C%BC%EB%A9%B0%2C+%EC%A3%BC%EB%AC%B8+%EA%B0%80%EB%8A%A5%ED%95%9C+%ED%94%84%EB%9E%9C%EC%B0%A8%EC%9D%B4%EC%A6%88+%EB%B8%8C%EB%9E%9C%EB%93%9C%EB%8A%94+38%EA%B0%9C%2C%0A++++%EA%B0%80%EB%A7%B9%EC%A0%90%EC%88%98%EB%8A%94+%EC%95%BD+1%EB%A7%8C+5%EC%B2%9C%EC%97%AC%EA%B3%B3%EC%97%90+%EB%8B%AC%ED%95%9C%EB%8B%A4.+%EC%A0%84+%EA%B5%AD%EB%AF%BC%EC%97%90%EA%B2%8C+%EC%B9%9C%EC%88%99%ED%95%9C+%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%86%A1+UI%EB%A5%BC+%ED%99%9C%EC%9A%A9%ED%95%98%EA%B8%B0+%EB%95%8C%EB%AC%B8%EC%97%90+%EB%82%A8%EB%85%80%EB%85%B8%EC%86%8C+%EB%88%84%EA%B5%

In [38]:
string = train_df['sentence'][0]

In [ ]:
from kakaotrans import Translator
Translator().translate("Try your best rather than be the best.",src = 'en',tgt = 'kr')
translator.translate("최고가 되기보다는 최선을 다하라.", src='kr', tgt='en')

In [ ]:
a = 'A América Latina localiza-se totalmente no hemisfério ocidental, sendo atravessada pelo Trópico de Câncer, que corta a parte central do México; pelo Equador, que passa pelo Brasil, Colômbia, Equador e toca o norte do Peru; e pelo Trópico de Capricórnio, que atravessa o Brasil, o Paraguai, a Argentina e o Chile.'

In [ ]:
b = '''
SENTENCE : A América Latina localiza-se totalmente no hemisfério ocidental, sendo atravessada pelo Trópico de Câncer, que corta a parte central do México; pelo Equador, que passa pelo Brasil, Colômbia, Equador e toca o norte do Peru; e pelo Trópico de Capricórnio, que atravessa o Brasil, o Paraguai, a Argentina e o Chile.

MANUALLY CHECKED : TRUE

ENTITY1 : América Latina
TYPE1 : LOCATION
ENTITY2 : Argentina
TYPE2 : LOCATION
REL TYPE : locatedInArea'''

In [10]:
import requests

def translate1(query, before_lang, after_lang):
    url = "https://translate.kakao.com/translator/translate.json"

    headers = {
        "Referer": "https://translate.kakao.com/",
        "User-Agent": "Mozilla/5.0"
    }

    data = {
        "queryLanguage": before_lang,
        "resultLanguage": after_lang,
        "q": query
    }

    resp = requests.post(url, headers=headers, data=data)
    data = resp.json()
    output = data['result']['output'][0][0]
    return output


if __name__ == "__main__":
    pass
#     print(translate(a))
#     print(translate(b))

In [ ]:
import os
import sys
import urllib.request
client_id = "cyeRpBs2NsnOJeI0XXef" # 개발자센터에서 발급받은 Client ID 값
client_secret = "CXaUQ43ejs" # 개발자센터에서 발급받은 Client Secret 값

encText = urllib.parse.quote("반갑습니다")
before_lang = input()
after_lang = input()
data = "source=" + before_lang + "&target=" + after_lang + "&text=" + encText
url = "https://openapi.naver.com/v1/papago/n2mt"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

def translate2(encText, before_lang, after_lang):
    data = "source=" + before_lang + "&target=" + after_lang + "&text=" + encText
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

In [35]:
li = []
for _ in range(len(train_df)):
    try:
        a = translate(train_df['sentence'][_],'en','kr')
        li.append(a)
    except:
        break

In [36]:
li[0]

IndexError: list index out of range

In [25]:
li = np.array(li)

In [28]:
data1 = pd.DataFrame(li)

In [29]:
data1

,0
0,"KakaoTalk, which opened in March last year, ha..."
1,"KakaoTalk, which opened in March last year, ha..."
2,"KakaoTalk, which opened in March last year, ha..."
3,"KakaoTalk, which opened in March last year, ha..."
4,"KakaoTalk, which opened in March last year, ha..."
...,...
91,"KakaoTalk, which opened in March last year, ha..."
92,"KakaoTalk, which opened in March last year, ha..."
93,"KakaoTalk, which opened in March last year, ha..."
94,"KakaoTalk, which opened in March last year, ha..."


In [27]:
np.save('./x_save', li)

# 출처: https://rfriend.tistory.com/358 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]

In [13]:
train_df['sentence'][:100]

0     The system as described above has its greatest...
1     The <e1>child</e1> was carefully wrapped and b...
2     The <e1>author</e1> of a keygen uses a <e2>dis...
3     A misty <e1>ridge</e1> uprises from the <e2>su...
4     The <e1>student</e1> <e2>association</e2> is t...
                            ...                        
95    The <e1>driver</e1> of the cement truck cleane...
96    The Marshall Plan also provided a way of gaini...
97    In the corner there are several gate captains ...
98    Once they grow there, the <e1>swelling</e1> an...
99    Guitars are played acoustically; the <e1>tone<...
Name: sentence, Length: 100, dtype: object

In [ ]:
aaa = train_df['sentence'].map(lambda x : translate1(x,'en','kr'))

In [ ]:
aaa